In [161]:
#dependencies
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import time
import pandas as pd

In [162]:
#configure splinter
executable_path = {'executable_path': 'C:/Users/chamb/Documents/chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [163]:
#define url to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

# Create BeautifulSoup object; parse with 'html'
html = browser.html
soup = bs(html, 'html.parser')

In [164]:
#find latest Mars article title
news_title = soup.find('div', class_="content_title").text
print(news_title)



NASA Announces Landing Site for Mars 2020 Rover




In [165]:
#find latest news paragraph
news_p = soup.find('div', class_="rollover_description_inner").text
print(news_p)


After a five-year search, NASA has chosen Jezero Crater as the landing site for its upcoming Mars 2020 rover mission.



In [166]:
#get JPL featured image
url_jpl = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url_jpl)

In [167]:
#access full image size
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(2)
browser.click_link_by_partial_text('more info')
time.sleep(2)
browser.click_link_by_partial_text('.jpg')

html = browser.html
soup = bs(html, 'html.parser')

featured_img_url = soup.find('img').get('src')
print(featured_img_url)

https://photojournal.jpl.nasa.gov/jpeg/PIA17936.jpg


In [168]:
#retrieve Mars weather from twitter account
url_weather = "https://twitter.com/marswxreport?lang=en"
browser.visit(url_weather)

In [169]:
html_weather = browser.html
soup = bs(html_weather, "html.parser")

In [170]:
#get latest tweets
tweets = soup.find_all('div', class_='js-tweet-text-container')

#only get latest tweet that mentions weather-related items
for tweet in tweets: 
    weather_tweet = tweet.find('p').text.replace('\n',' ').split('pic')[0]
    if 'Sol' and 'winds' and 'pressure' in weather_tweet:
        print(weather_tweet)
        break
    else: 
        pass

InSight sol 132 (2019-04-10) low -97.7ºC (-143.8ºF) high -16.3ºC (2.7ºF) winds from the SW at 4.3 m/s (9.5 mph) gusting to 12.2 m/s (27.3 mph) pressure at 7.30 hPa


In [58]:
#use pandas to scrape tabular data from mars facts site
url = 'https://space-facts.com/mars/'

In [60]:
tables = pd.read_html(url)

In [61]:
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [154]:
type(tables)

list

In [63]:
#convert to pandas dataframe
df = tables[0]
df.columns = ['Attribute','Value']

In [64]:
df

,Attribute,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [156]:
df.to_html(index=False)

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Attribute</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

In [127]:
#define main url of site pertaining to the hemispheres of Mars
main_url = 'https://astrogeology.usgs.gov'

In [151]:
#navigate to page on Mars hemispheres
url_hem = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url_hem)
    
html_hemispheres = browser.html
soup = bs(html_hemispheres, 'html.parser')

#create list of hemisphere titles
items = soup.find_all('div', class_='item')

In [152]:
#create empty list to hold hemisphere title and image url
hemisphere_images = []

#loop over hemispheres and save the titles and image links
for item in items:
    url_hem = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    browser.visit(url_hem)
    
    html_hemispheres = browser.html
    soup = bs(html_hemispheres, 'html.parser')
    
    title = item.find('h3').text
    
    #navigate to specific hemisphere page
    browser.click_link_by_partial_text(title)
    time.sleep(2)

    html = browser.html
    soup = bs(html, 'html.parser')
    
    #construct link to hemisphere image
    partial_img_url = soup.find('img', class_='wide-image')['src']
    img_url = main_url + partial_img_url

    #append hemisphere title and image url to list
    hemisphere_images.append({"title":title,"img_url":img_url})

In [153]:
print(hemisphere_images)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url=': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url=': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url=': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url=': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]
